In [ ]:
%matplotlib inline


# essai

on tente des trucs


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from matplotlib import pyplot as plt

from py_eddy_tracker.observations.network import NetworkObservations


def rect_from_extent(extent):
    rect_lon = [extent[0], extent[1], extent[1], extent[0], extent[0]]
    rect_lat = [extent[2], extent[2], extent[3], extent[3], extent[2]]
    return rect_lon, rect_lat


def indice_from_extent(lon, lat, extent):
    mask = (lon > extent[0]) * (lon < extent[1]) * (lat > extent[2]) * (lat < extent[3])
    return np.where(mask)[0]


fichier = "/data/adelepoulle/work/Eddies/20201217_network_build/big_network.nc"
network = NetworkObservations.load_file(fichier)
sub_network = network.network(1078566)

In [ ]:
# extent_begin = [0, 2, -50, -15]
# extent_end = [-42, -35, -40, -10]

extent_begin = [2, 22, -50, -30]
i_obs_begin = indice_from_extent(
    sub_network.longitude, sub_network.latitude, extent_begin
)
network_begin = sub_network.find_link(i_obs_begin)
time_mini = network_begin.time.min()
time_maxi = network_begin.time.max()

extent_end = [-52, -45, -37, -33]
i_obs_end = indice_from_extent(
    (network_begin.longitude + 180) % 360 - 180, network_begin.latitude, extent_end
)
network_end = network_begin.find_link(i_obs_end, forward=False, backward=True)


datasets = [network_begin, network_end]
extents = [extent_begin, extent_end]
fig, (ax1, ax2) = plt.subplots(
    2, 1, figsize=(10, 9), dpi=140, subplot_kw={"projection": ccrs.PlateCarree()}
)

for ax, dataset, extent in zip([ax1, ax2], datasets, extents):
    sca = dataset.scatter(
        ax,
        name="time",
        cmap="Spectral_r",
        label="observation dans le temps",
        vmin=time_mini,
        vmax=time_maxi,
    )

    x, y = rect_from_extent(extent)
    ax.fill(x, y, color="grey", alpha=0.3, label="observations choisies")
    # ax.plot(x, y, marker='o')

    ax.legend()

    gridlines = ax.gridlines(
        alpha=0.2, color="black", linestyle="dotted", draw_labels=True, dms=True
    )

    gridlines.left_labels = False
    gridlines.top_labels = False

    ax.coastlines()
    ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.LAKES, zorder=10)
    ax.add_feature(cfeature.BORDERS, lw=0.25)
    ax.add_feature(cfeature.OCEAN, alpha=0.2)


ax1.set_title(
    "Recherche du déplacement de l'eau dans les eddies à travers les observations choisies"
)
ax2.set_title("Recherche de la provenance de l'eau à travers les observations choisies")
ax2.set_extent(ax1.get_extent(), ccrs.PlateCarree())

fig.subplots_adjust(right=0.87, left=0.02)
cbar_ax = fig.add_axes([0.90, 0.1, 0.02, 0.8])
cbar = fig.colorbar(sca["scatter"], cax=cbar_ax, orientation="vertical")
_ = cbar.set_label("time (jj)", rotation=270, labelpad=-65)